<div style = "background-image:'conda.jpg';background-repeat: no-repeat">


<center> 

<h1 style="color:blue"> Kaggle Expedia Hotel Recommendation Competition</h1>
<h2 style = "color:orange"> Team Snakes </h2>
<h3>
Hardy Chen<br>
Xing Gao<br>
Brandon Hart<br>
Jenny Ma<br>
</h3>

</center>

<br>
<br>
<br>
<br>
<br>
<br>
<hr>

<div style = "background-color:orange;padding:5px;border:solid;border-color:black;">
<p>Expedia wants to provide personalized hotel recommendations to their users. </p>
<p>https://www.kaggle.com/c/expedia-hotel-recommendations</p>
</div>
</div>

<img src = "expedia.png">

<center><img src = "leaders.jpg"></center>

In [10]:
import pandas as pd


train = pd.read_csv("train.csv") # Almost 4 Gigs!!!
test = pd.read_csv("test.csv")

# See how many rows/columns 
print(train.shape) # includes website click data from 2013 and 2014
print(test.shape) # includes actual booking in 2015

# Target attribute is hotel_cluster, which isn't given in test data set
# We must give 5 predictions, so expedia can show new user 5 hotel clusters
# based on their activity on the website

# Get familiar with data
train.head(5)
test.head(5)

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


<h1><blink>Yikes, that's a lot of data!</blink></h1>

In [ ]:
sampleTraining = training.head(5)
print (sampleTraining)

In [12]:
#distribution of hotel
hotelDistribution = train["hotel_cluster"].value_counts()
print (hotelDistribution)
# Pretty even

91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, dtype: int64


In [13]:
#ids in test dataset is subset of ids in train dataset
#so when we downsample, we need to keep this rule

testids = set(test.user_id.unique())
trainids = set(train.user_id.unique())

inter = len(testids & trainids)
inter == len(testids)

True

In [18]:
#since date_time is string in original dataset, we convert it to year and month
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year # Creating Column Year and extracting year from date_time
train["month"] = train["date_time"].dt.month # Creating Column Month and extracting month from date_time

print(train.head(5))

            date_time  site_name  posa_continent  user_location_country  \
0 2014-08-11 07:46:59          2               3                     66   
1 2014-08-11 08:22:12          2               3                     66   
2 2014-08-11 08:24:33          2               3                     66   
3 2014-08-09 18:05:16          2               3                     66   
4 2014-08-09 18:08:18          2               3                     66   

   user_location_region  user_location_city  orig_destination_distance  \
0                   348               48862                  2234.2641   
1                   348               48862                  2234.2641   
2                   348               48862                  2234.2641   
3                   442               35390                   913.1932   
4                   442               35390                   913.6259   

   user_id  is_mobile  is_package  ...    srch_destination_id  \
0       12          0           1  ... 

In [19]:
#downsample to 10000 users 
import random
unique_users = train.user_id.unique()

select_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)),10000))]
select_train = train[train.user_id.isin(select_user_ids)]
                     
train10000 = select_train[((select_train.year == 2013) | ((select_train.year == 2014) & (select_train.month < 8)))]
test10000 = select_train[((select_train.year == 2014) & (select_train.month >= 8))]

In [20]:
#remove click event
#because we use test dataset to find out if our prediction correcr or not, so we only need row which have booking already                     
test10000 = test10000[test10000.is_booking == True]

In [21]:
#do a simple test algorithm: [top 5 popular] head defaults to 5 and index sorts Desc
#so we can understand how the game works
mostCommonCluster = list(train10000.hotel_cluster.value_counts().head().index)
print(mostCommonCluster)


[91, 41, 48, 64, 5]


In [22]:
predictions = []
for i in range(test10000.shape[0]):
    predictions.append(mostCommonCluster)

In [24]:
import ml_metrics as metrics
target = [[l] for l in test10000["hotel_cluster"]]
target = []
for l in test10000["hotel_cluster"]:
    target.append([l])
metrics.mapk(target, predictions, k=5)


0.062708825847123714

In [25]:
#Use srch_destination_id to generate top clusters in each detsination 
#clusterCols = [["srch_destination_id"],["hotel_cluster"]]
#groups = {(destination, hotel):[row1,row2],(destination, hotel):[row3,row4]}

matchCols = ["srch_destination_id"]
clusterCols = matchCols + ["hotel_cluster"] #Concatenates
groups = train10000.groupby(clusterCols) # Aggregates


In [27]:
#score algorithm: score = bookings + 0.25 * clicks to calculate the most popular hotel cluster
topClusters = {}
# looks like this groups = {(destination, hotel):[row1,row2],(destination, hotel):[row3,row4]}
for name,group in groups:
    clicks = len(group.is_booking[group.is_booking == 0])
    bookings = len(group.is_booking[group.is_booking == 1])
    score = bookings + 0.25 * clicks
    
    clusName = name[0]
    if clusName not in topClusters:
        topClusters[clusName] = {}
    topClusters[clusName][name[-1]] = score

In [35]:
#clusName = make_key(name[:len(matchCols)])
#{'destination':{'Hotel':socore}}  

import operator
clusterDict ={}
for n in topClusters:
    tc = topClusters[n]
    top= [ lst[0] for lst in sorted(tc.items(), key=operator.itemgetter(1),reverse=True)[:5]]
    clusterDict[n] = top    

In [36]:
#make a prediction
prediction = []
for index,row in test10000.iterrows():
    key = row["srch_destination_id"]
    if key in clusterDict:
        prediction.append(clusterDict[key])
    else:
        prediction.append([])
        
        
#evaluate our prediction
#we got accuracy 0.23256897169473309
metrics.mapk([[l] for l in test10000["hotel_cluster"]], prediction, k=5)



0.23507289204097714

In [37]:
#try better predictions
matchCols1 =  ['user_location_region', 'orig_destination_distance', 'hotel_market']
# find clusters in the created groups above row by row. The index will be the value of those four variables above in matchCols.
# Then we use this index to find the hotel clusters. If it exists, then the cluster value will be appended into the clus list.
# If the clusters value does not exsit, an empty list will be generate into clus.
def generate_exact_matches(row,matchCols):
    groups = train10000.groupby(matchCols)
    index = tuple()
    for t in matchCols:
        index += (row[t],)

    try:
        group = groups.get_group(index)
    except Exception:
        return []
    cluster = list(set(group.hotel_cluster))
    return cluster


In [ ]:
#Try our function
exactMatches1= []
for i in range(test10000.shape[0]):
    exactMatches1.append(generate_exact_matches(test10000.iloc[i],matchCols1))

In [ ]:
import ml_metrics as metrics
target = [[l] for l in test10000["hotel_cluster"]]
metrics.mapk([[l] for l in test10000["hotel_cluster"]], exactMatches1, k=5)